In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import torch
from torch import nn
from fastcore.foundation import L
from fastai.data.transforms import TfmdLists, DataLoaders, RandomSplitter, ToTensor
from fastai.losses import MSELossFlat
from fastai.learner import Learner
import fastai.callback.schedule   # Needed for fit_one_cycle

import custom_layers

In [ ]:
def generate_data_for_one_feature_gauss_test(N:int, sample_len:int, noise:bool=True):
    """
    Generate a bunch of sinus signals with various frequencies and a bit of noise.

    Returns an array of size N x sample_len
    """
    rnd_offsets = np.random.rand(N) * np.pi
    rnd_frequencies = np.random.randint(1, 3, N)
    signal = np.zeros((N, sample_len), np.float32)
    for k in range(N):
        signal[k, :] = np.linspace(0,  rnd_frequencies[k] * 2 * np.pi, sample_len) - rnd_offsets[k]
        signal[k, :] = np.sin(signal[k, :])
    if noise:
        signal *= np.random.rand(N, sample_len)
    return signal.astype(np.float32)


def gen_gauss(x_arr, sigma, miu):
    """
    Generates a Gaussian kernel with support x_arr and (miu, sigma) stats.
    """
    x_arr_shift = x_arr - miu
    gauss_kern = np.exp(-np.square(x_arr_shift) / (2*sigma*sigma))
    gauss_kern = gauss_kern * 1 / (sigma * np.sqrt(2*np.pi))

    return gauss_kern

In [ ]:
np.random.seed(343)
sample_len = 100
target_sigma = 3
target_mu = -11.5

signal = generate_data_for_one_feature_gauss_test(1, sample_len, True)  # A demo signal
signal = signal[0]

nk = sample_len - (1 - sample_len % 2)
kernel_support = np.arange(-nk/2, nk/2, 1)
kernel = gen_gauss(kernel_support, target_sigma, target_mu)  # A Gauss kernel
kernel_zero_shift = gen_gauss(kernel_support, target_sigma, 0)  # A Gauss kernel with 0 shift
filtered_sample = np.convolve(signal, kernel, mode='same')
noshift_filter_sample = np.convolve(signal, kernel_zero_shift, mode='same')


In [ ]:
fig, ax = plt.subplots(3,1, figsize=(8,6))
ax[0].plot(signal, label="Original signal")
ax[0].set_ylim([-1, 1])
ax[0].plot(noshift_filter_sample, label="Smoothed signal")
ax[0].legend()
ax[0].set_title("Some sinusoid signal with noise")
ax[1].plot(kernel_support, kernel)
ax[1].set_title("The Gauss kernel, with non zero mean")
ax[2].plot(filtered_sample, label="Transformed signal")
ax[2].plot(noshift_filter_sample, label="Smoothed signal")
ax[2].set_ylim([-1, 1])
ax[2].legend()
ax[2].set_title("Filtered signal. Note the shift!")
fig.tight_layout()

Can a network learn this smoothing and shifting, just by looking at a bunch of samples?

Can our layer learn such a thing?

In [ ]:
def display_a_sample(tls_train, sample_id, model, title):
    sample = tls_train[sample_id][0]
    sample_t = torch.tensor(sample, device=torch.device('cuda')).unsqueeze(0)
    sample_gt = tls_train[sample_id][1]
    transformed_signal = model(sample_t).detach().squeeze(0).cpu().numpy()
    fig, ax = plt.subplots(2,1, figsize=(8,6))
    ax[0].plot(sample, label="Original signal")
    ax[0].set_title("Data sample")
    ax[1].plot(transformed_signal, label="Network output")
    ax[1].plot(sample_gt, label="GT")
    ax[1].legend()
    ax[1].set_title("Transformed data")
    fig.suptitle(title)
    fig.tight_layout()

In [ ]:
N = 500
sample_id = 34  # We will watch this sample
signals = generate_data_for_one_feature_gauss_test(N, sample_len, True)  # A bunch of signals
gt_signals = []
for k in range(N):
    gt_sample = np.convolve(signals[k, :], kernel, mode='same')
    gt_signals.append(gt_sample)
gt_signals = np.array(gt_signals).astype(np.float32)
train_samples = zip(signals, gt_signals)
train_samples = L(train_samples)

splits = RandomSplitter(0.1)(train_samples)
tls_train = TfmdLists(train_samples[splits[0]], [ToTensor()])
tls_test = TfmdLists(train_samples[splits[1]], [ToTensor()])
dloaders = DataLoaders.from_dsets(tls_train, tls_test, bs=1, num_workers=4, device=torch.device('cuda'))
model = custom_layers.GaussConvLayer_nobackwards().to(torch.device('cuda'))
learner = Learner(dloaders, model, loss_func=MSELossFlat(reduction="mean"), model_dir="../dump")
display_a_sample(tls_train, sample_id, model,"Model after the initialization")

In [ ]:
learner.fit_one_cycle(3, 1e-2)
display_a_sample(tls_train, sample_id, model,"Model after 3 epochs")

In [ ]:
learner.fit_one_cycle(3, 1e-2)
display_a_sample(tls_train, sample_id, model,"Model after 6 epochs")